In [10]:
import time
import numpy as np
import pandas as pd
import keras

from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.models import Model
from keras.layers.merge import dot
from keras.optimizers import Adam
from keras.layers import Dense , merge
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split

In [11]:
np.random.seed(123)
# load data
def loadData():
    ratings = pd.read_csv('./data/rating.csv', parse_dates=['timestamp'])
    return ratings

In [12]:
# for test convenience, only use num% of data
def cutData(num, ratings):
    rand_userIds = np.random.choice(ratings['userId'].unique(), size=int(len(ratings['userId'].unique())*num), replace=False)
    ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]
    return ratings

In [13]:
def splitData(ratings):
    users = ratings.userId.unique()
    movies = ratings.movieId.unique()

    userid2idx = {o:i for i,o in enumerate(users)}
    movieid2idx = {o:i for i,o in enumerate(movies)}

    ratings['userId'] = ratings['userId'].apply(lambda x: userid2idx[x])
    ratings['movieId'] = ratings['movieId'].apply(lambda x: movieid2idx[x])
    train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)
    
    print(train_ratings.shape , test_ratings.shape)
    return train_ratings, test_ratings

In [14]:
def embeddingNNModel(ratings):
    n_movies=len(ratings['movieId'].unique())
    n_users=len(ratings['userId'].unique())
    n_latent_factors=50  # hyperparamter to deal with. 

    user_input=Input(shape=(1,),name='user_input',dtype='int64')
    user_embedding=Embedding(n_users,n_latent_factors,name='user_embedding')(user_input)
    user_vec =Flatten(name='FlattenUsers')(user_embedding)
    user_vec=Dropout(0.40)(user_vec)

    movie_input=Input(shape=(1,),name='movie_input',dtype='int64')
    movie_embedding=Embedding(n_movies,n_latent_factors,name='movie_embedding')(movie_input)
    movie_vec=Flatten(name='FlattenMovies')(movie_embedding)
    movie_vec=Dropout(0.40)(movie_vec)

    sim=dot([user_vec,movie_vec],name='Simalarity-Dot-Product',axes=1)
    nn_inp=Dense(96,activation='relu')(sim)
    nn_inp=Dropout(0.4)(nn_inp)
    nn_inp=Dense(1,activation='relu')(nn_inp)
    nn_model =keras.models.Model([user_input, movie_input],nn_inp)
    return nn_model

In [15]:
def fit(model, train_ratings, epochs, batch_size):
    model.compile(optimizer=Adam(lr=1e-4),loss='mse')
    model.fit([train_ratings.userId,train_ratings.movieId], train_ratings.rating, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

In [16]:
def predict(model, test_ratings):
    pre_ratings = model.predict([test_ratings.userId,test_ratings.movieId])
    return pre_ratings

In [17]:
def rmse(prediction, ground_truth):
    return sqrt(mean_squared_error(prediction, ground_truth))

In [18]:
start = time.time()
ratings = loadData()
ratings_cut = cutData(0.05, ratings)
train_ratings, test_ratings = splitData(ratings_cut)
nn_model = embeddingNNModel(ratings_cut)
batch_size = 512
epochs = 10
new_model = fit(nn_model, train_ratings, epochs, batch_size)
pre_ratings = predict(new_model, test_ratings)
RMSE = rmse(pre_ratings, test_ratings.rating)
print (f'Batch_size is: {batch_size}, epochs is: {epochs}, Neural Network RMSE is: {RMSE}')
end = time.time()
print(f"Runtime of the program is {end - start}")

<ipython-input-13-e6ff1afe7310>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['userId'] = ratings['userId'].apply(lambda x: userid2idx[x])
<ipython-input-13-e6ff1afe7310>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['movieId'] = ratings['movieId'].apply(lambda x: movieid2idx[x])


(814383, 4) (203596, 4)
Epoch 1/10
1591/1591 [==============================] - 6s 4ms/step - loss: 10.4317
Epoch 2/10
1591/1591 [==============================] - 7s 4ms/step - loss: 1.5141
Epoch 3/10
1591/1591 [==============================] - 7s 5ms/step - loss: 1.2709
Epoch 4/10
1591/1591 [==============================] - 6s 4ms/step - loss: 1.2611
Epoch 5/10
1591/1591 [==============================] - 7s 4ms/step - loss: 1.2463
Epoch 6/10
1591/1591 [==============================] - 7s 4ms/step - loss: 1.1759
Epoch 7/10
1591/1591 [==============================] - 7s 4ms/step - loss: 1.0805
Epoch 8/10
1591/1591 [==============================] - 7s 4ms/step - loss: 0.9945
Epoch 9/10
1591/1591 [==============================] - 7s 4ms/step - loss: 0.9377
Epoch 10/10
1591/1591 [==============================] - 7s 4ms/step - loss: 0.8959
Batch_size is: 512, epochs is: 10, Neural Network RMSE is: 0.9016404685191259
Runtime of the program is 113.98318696022034
